# Imports

In [23]:
import pandas as pd

In [46]:
# Import the dataset into a pandas DataFrame
df = pd.read_csv('github_repos_completos.csv')

df.head()

,name,owner,stars,forks,language,created_at,updated_at,size_kb,watchers_count,open_issues,owner_type,owner_public_repos,owner_location,subscribers_count,last_year_commits,contributors,closed_issues,pull_requests
0,free-programming-books,EbookFoundation,359735,63576,Python,2013-10-11T06:50:37Z,2025-06-21T02:09:07Z,19483,359735,45,Organization,37,virtual,5000,339,432,129,15
1,public-apis,public-apis,351991,37004,Python,2016-03-20T23:49:42Z,2025-06-21T02:06:40Z,4771,351991,492,Organization,1,NaN,4316,5,427,159,490
2,system-design-primer,donnemartin,306925,50727,Python,2017-02-26T16:15:28Z,2025-06-21T02:06:11Z,11239,306925,498,User,27,"Washington, D.C.",5000,4,113,18,256
3,awesome-python,vinta,247255,25843,Python,2014-06-27T21:00:06Z,2025-06-21T02:08:14Z,6769,247255,486,User,20,Taiwan,5000,1,368,28,471
4,Python,TheAlgorithms,201541,46909,Python,2016-07-16T09:44:01Z,2025-06-21T01:50:56Z,15391,201541,397,Organization,44,India,5000,181,454,334,329


# Data Quality Analysis

## Missing Data

In [47]:
# Check for and sum the number of missing (null) values for each column.
df.isnull().sum()

,0
name,0
owner,0
stars,0
forks,0
language,1
created_at,0
updated_at,0
size_kb,0
watchers_count,0
open_issues,0


In [48]:
# Calculate and print the proportion of null values for 'language' and 'owner_location' columns.
language_null_prop = (df['language'].isnull().sum() / len(df)) * 100
location_null_prop = (df['owner_location'].isnull().sum() / len(df)) * 100

print('Proporção de nulos:')
print(f'Atributo language: {language_null_prop:.2f}%')
print(f'Atributo owner_location: {location_null_prop:.2f}%')

Proporção de nulos:
Atributo language: 0.01%
Atributo owner_location: 42.00%


In [49]:
# Handle missing values:
df = df[df['language'].notnull()]

df['owner_location'].fillna('Not informed', inplace=True)

/tmp/ipython-input-49-474970543.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['owner_location'].fillna('Not informed', inplace=True)
/tmp/ipython-input-49-474970543.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['owner_location'].fillna('Not informed', inplace=True)


In [50]:
df.isnull().sum()

,0
name,0
owner,0
stars,0
forks,0
language,0
created_at,0
updated_at,0
size_kb,0
watchers_count,0
open_issues,0


## Duplicated Data

In [51]:
# Identify and count completely duplicate rows in the DataFrame.
df_duplicated = df[df.duplicated()]

print(f"Quantidade de duplicatas completas: {df_duplicated.shape[0]}")

Quantidade de duplicatas completas: 348


In [54]:
df_duplicated['language'].value_counts()

,count
language,
C,347
Java,1


In [55]:
# Remove all identified duplicate rows from the DataFrame.
df = df.drop_duplicates()

print(f'Após exclusão de dados duplicados: {len(df)} registros')

Após exclusão de dados duplicados: 9101 registros


## Inconsistent Data

In [56]:
# Identify and list all numeric columns.
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Iterate through numeric columns to check for and display negative values.
for column in numeric_columns:
    negativos = df[df[column] < 0]
    qtd = negativos.shape[0]
    if qtd > 0:
        print(f"\nColuna '{column}' tem {qtd} valores negativos:")
        print(negativos[[column]].head(10))
    else:
        print(f"Coluna '{column}' não possui valores negativos.")

Coluna 'stars' não possui valores negativos.
Coluna 'forks' não possui valores negativos.
Coluna 'size_kb' não possui valores negativos.
Coluna 'watchers_count' não possui valores negativos.
Coluna 'open_issues' não possui valores negativos.
Coluna 'owner_public_repos' não possui valores negativos.
Coluna 'subscribers_count' não possui valores negativos.
Coluna 'last_year_commits' não possui valores negativos.
Coluna 'contributors' não possui valores negativos.
Coluna 'closed_issues' não possui valores negativos.
Coluna 'pull_requests' não possui valores negativos.


In [58]:
# Ensuring date columns are in datetime format
for col in ['created_at', 'updated_at']:
    df[col] = pd.to_datetime(df[col], errors='coerce')

    n_nulos = df[col].isna().sum()
    print(f"Coluna '{col}' após conversão para datetime tem {n_nulos} valores que não foram convertidos")

Coluna 'created_at' após conversão para datetime tem 0 valores que não foram convertidos
Coluna 'updated_at' após conversão para datetime tem 0 valores que não foram convertidos


In [59]:
# Checking for non-numeric entries in numeric columns
for col in numeric_columns:
    coerced = pd.to_numeric(df[col], errors='coerce')
    n_invalidos = coerced.isna().sum()
    print(f"Coluna '{col}' tem {n_invalidos} valores não numéricos (NaN após conversão)")

Coluna 'stars' tem 0 valores não numéricos (NaN após conversão)
Coluna 'forks' tem 0 valores não numéricos (NaN após conversão)
Coluna 'size_kb' tem 0 valores não numéricos (NaN após conversão)
Coluna 'watchers_count' tem 0 valores não numéricos (NaN após conversão)
Coluna 'open_issues' tem 0 valores não numéricos (NaN após conversão)
Coluna 'owner_public_repos' tem 0 valores não numéricos (NaN após conversão)
Coluna 'subscribers_count' tem 0 valores não numéricos (NaN após conversão)
Coluna 'last_year_commits' tem 0 valores não numéricos (NaN após conversão)
Coluna 'contributors' tem 0 valores não numéricos (NaN após conversão)
Coluna 'closed_issues' tem 0 valores não numéricos (NaN após conversão)
Coluna 'pull_requests' tem 0 valores não numéricos (NaN após conversão)


In [61]:
# Identify categorical columns.
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

# For each categorical column, print unique values and their normalized counts.
for col in categorical_columns:
    print(f"\nValores únicos e suas contagens na coluna '{col}':")
    valores_unicos = df[col].dropna().unique()

    # Normalizing to lower case and removing spaces to identify variations.
    valores_normalizados = [str(v).strip().lower() for v in valores_unicos]
    contagem = pd.Series(valores_normalizados).value_counts()

    print(contagem.head(20))


Valores únicos e suas contagens na coluna 'name':
leetcode      3
tv            2
pulse         2
cameraview    2
icecream      2
gifski        2
menu          2
eureka        2
cardslider    2
java          2
schedule      2
ignite        2
time          2
surge         2
glance        2
nuklear       2
iris          2
shadow        2
skip          2
ios           2
Name: count, dtype: int64

Valores únicos e suas contagens na coluna 'owner':
stasel                  1
ebookfoundation         1
public-apis             1
donnemartin             1
vinta                   1
thealgorithms           1
significant-gravitas    1
automatic1111           1
cocoapods               1
hearthsim               1
xjbeta                  1
slazyk                  1
marioiannotta           1
radex                   1
kaandedeoglu            1
alexeybelezeko          1
yeahdongcn              1
venmo                   1
jiritrecak              1
pixel16                 1
Name: count, dtype: int64

Valo

In [68]:
unique_locations = df['owner_location'].dropna().unique()

unique_location_normalized = [str(v).strip().lower() for v in unique_locations]
contagem_df = pd.Series(unique_location_normalized).value_counts().reset_index()
contagem_df.columns = ['location', 'count']

contagem_df.to_csv('location_counts.csv', index=False)

In [71]:
def limpar_localizacao(loc):
    if pd.isna(loc):
        return loc
    loc = loc.lower().strip()
    loc = loc.replace(',', '')
    loc = loc.replace('"', '')
    termos_genericos = ['earth', 'internet', 'the internet', 'world', 'worldwide', 'interwebs', 'the cloud',
                        'github', 'all around the world', 'virtual', 'anywhere', 'in cloud', 'git earth',
                        'wildest dream', 'crazy universe']

    if loc in termos_genericos:
        return 'not informed'
    return loc

In [73]:
print(df['owner_location'].value_counts().head(30))

owner_location
not informed                3785
united states of america     402
china                        183
Not informed                 138
san francisco ca             117
san francisco                109
hangzhou china               100
germany                       87
beijing china                 86
shanghai china                65
tokyo japan                   64
singapore                     64
redmond wa                    60
london uk                     53
france                        52
united kingdom                49
shanghai                      48
london                        47
beijing                       43
menlo park california         41
paris france                  41
shenzhen china                41
netherlands                   40
seattle wa                    40
new york ny                   35
berlin                        34
stockholm sweden              34
india                         33
new york                      30
canada                      

In [ ]:
# def mapear_localizacao(loc):
#     if pd.isna(loc):
#         return 'desconhecido'

#     loc = loc.lower().strip()
#     loc = loc.replace(',', '')

#     # Normalizar genéricos
#     termos_desconhecidos = ['earth', 'internet', 'the internet', 'world', 'unknown']
#     if loc in termos_desconhecidos:
#         return 'desconhecido'

#     # Agrupar por país/região
#     if any(cidade in loc for cidade in ['beijing', 'hangzhou', 'shenzhen', 'shanghai', 'guangdong', 'china']):
#         return 'china'
#     elif any(cidade in loc for cidade in ['new york', 'sunnyvale', 'santa monica', 'mountain view', 'seattle', 'usa', 'united states', 'ca']):
#         return 'usa'
#     elif 'munich' in loc or 'germany' in loc:
#         return 'germany'
#     elif 'singapore' in loc:
#         return 'singapore'

#     # Se não cair em nenhuma regra, retorna a string limpa
#     return loc

# # Aplicar a função
# df['owner_location'] = df['owner_location'].apply(mapear_localizacao)

# # Conferir as novas categorias agrupadas
# print(df['owner_location'].value_counts().head(20))